In [6]:
import os
import psycopg2
import pandas as pd
from transformers import BertModel, BertJapaneseTokenizer

# データベース接続設定
db_connection_string = "postgresql://{}:{}@postgres-service:{}/{}".format(
    os.environ['POSTGRES_USER'], 
    os.environ['POSTGRES_PASSWORD'], 
    os.environ['POSTGRES_SERVICE_SERVICE_PORT'], 
    os.environ['POSTGRES_DB'])

# データベースからデータを読み込む
with psycopg2.connect(db_connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT title FROM learning_resources')
        records = cur.fetchall()
        # レコードをテキストリストとして取得
        texts = [record[0] for record in records]
        
# 日本語のBERTモデルとトークナイザーの読み込み
model_name = "cl-tohoku/bert-base-japanese"
model = BertModel.from_pretrained(model_name)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# トークナイズと前処理の実行（最大長を指定）
encoded_texts = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=512,  # 最大トークン長を指定
    return_tensors="pt"
)

print(encoded_texts)

{'input_ids': tensor([[    2,  8037, 23525,  ...,     0,     0,     0],
        [    2,  3846,  6813,  ...,     0,     0,     0],
        [    2,     1, 23531,  ...,     0,     0,     0],
        ...,
        [    2, 20312, 28566,  ...,     0,     0,     0],
        [    2, 18505, 28521,  ...,     0,     0,     0],
        [    2,  7446,  8599,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# 事前学習済みモデルとトークナイザの読み込み
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# ドメイン特化データセットの読み込み（例：医療関連のテキスト）
# 'path_to_your_domain_data.txt' は、ファインチューニングに使用するテキストファイルへのパスです
dataset = load_dataset("text", data_files={"train": "path_to_your_domain_data.txt"})

# データセットの前処理
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

train_dataset = dataset["train"].map(preprocess_function, batched=True)

# トレーニングの設定
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
)

# トレーナーの初期化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# ファインチューニングの実行
trainer.train()
